In [ ]:
Investor_name = "JSK"

In [ ]:
# Imports
import pandas as pd
from nsetools import Nse
pd.set_option('display.float_format', lambda x: '%.3f' % x)


def get_data(filename="./data/Investments.csv", sheetname="Common", type="csv"):
    if type == "csv":
        return pd.read_csv(filename)
    return pd.read_excel(filename, sheet_name=sheetname, engine="openpyxl")


def convert_amount(x):
    if x['Transaction Type'] == "Investment":
        return x['Amount']
    return -1*x['Amount']


def convert_quantity(x):
    if x['Transaction Type'] == "Investment":
        return x['Sub-type quantity']
    return -1*x['Sub-type quantity']

def get_stock_live_price(symbol):
    try:
        return round(float(nse.get_quote(symbol)['lastPrice']), 2)
    except Exception as e:
        print("Couldn't fetch live price for a stock.", e)
        return 0
    

def name_to_symbol(name):
    try:
        return get_stock_live_price(s_dict[name])
    except Exception as e:
        print("Coulnd't find symbol for:", e)
        return 0


# Initialize
nse = Nse()

# Get data
df = get_data()
df.dropna(axis=1, how='all', inplace=True)

# Get stocks data
stock_names_df = pd.read_csv("./data/Stock_Symbols_NSE.csv")
stock_names_df.dropna(inplace=True)
s_names = [x.lower().strip() for x in stock_names_df['Company Name']]
s_symbol = [x.lower().strip() for x in stock_names_df['Symbol']]
s_dict = dict()
for k, v in zip(s_names, s_symbol):
    s_dict[k] = v

# Convert amount based on trasaction type
df["Amount"] = df[["Amount", "Transaction Type"]].apply(lambda x: convert_amount(x), axis=1)
# Convert quantity based on trasaction type
df["Sub-type quantity"] = df[["Sub-type quantity", "Transaction Type"]].apply(lambda x: convert_quantity(x), axis=1)

# Convert USD to INR
dollar_rate = 74.38
df['Amount_INR'] = df['Amount']
df['Amount_INR'] = df[df['Currency']=='USD']['Amount'].apply(lambda x: x*dollar_rate)
df.Amount_INR.fillna(df.Amount, inplace=True)

## Get Details!!

In [ ]:
query1 = df[(df['Investor']==Investor_name) & (df['Type']=='Stocks')].groupby(['Investor', 'Sub-type']).sum()
print("Profit drawn already:", -1 * sum(query1[query1['Sub-type quantity'] == 0]['Amount_INR'].tolist()))
query1 = query1[query1['Sub-type quantity'] != 0]

query1

In [ ]:
query2 = df[(df['Investor']==Investor_name) & (df['Type']=='Crypto')].groupby(['Investor', 'Sub-type']).sum()
query2

In [ ]:
print("Total invested in Stocks:", sum(query1['Amount_INR'].tolist()))
print("Total invested in Crypto:", sum(query2['Amount_INR'].tolist()))
print("Total invested in Stocks & Crypto:", sum(query1['Amount_INR'].tolist())+sum(query2['Amount_INR'].tolist()))

In [ ]:
# All investors combined for each platform
df[(df['Type']=='Crypto') | (df['Type']=='Stocks')].groupby("Platform").sum().reset_index()[['Platform', 'Amount_INR']]

## Get current prices

## Stocks

In [ ]:
query1 = query1.reset_index()
query1['Stock_Live_Price'] = query1['Sub-type'].apply(lambda x: name_to_symbol(x.lower().strip()))
query1['Stock_Live_Price'] = query1['Stock_Live_Price'] * query1['Sub-type quantity']
query1

In [ ]:
invested_amount = round(sum(query1['Amount_INR'].tolist()), 2)
current_value = round(sum(query1['Stock_Live_Price'].tolist()), 2)
print("Total invested in Stocks:", invested_amount)
print("Total value in Stocks:", current_value)
try:
    print("Total profit/loss in Stocks: {0} \t({1}%)".format(current_value-invested_amount, round(((current_value-invested_amount)/invested_amount)*100, 2))) 
except Exception as e:
    print("")

In [ ]:
import json
import requests

prices_coins = requests.get('https://api.wazirx.com/api/v2/tickers')


def warizx_price_fetch(coin, quantity):
    market_value = json.loads(prices_coins.text)[str(coin).lower()+"inr"]['last']
    return float(market_value) * float(quantity)

prices = list()
for row in query2.reset_index().itertuples():
    try:
        prices.append(warizx_price_fetch(row[2], row[3]))
    except KeyError as e:
        prices.append(0)
        print("Skiping:", row[2])
print("Done!!\n\n------")
query2['Market_value'] = prices
query2.drop('Amount', axis=1, inplace=True)

invested_amount = round(sum(query2['Amount_INR'].tolist()), 2)
current_value = round(sum(query2['Market_value'].tolist()), 2)
print("Total invested in Crypto:", invested_amount)
print("Total value in Crypto:", current_value)
print("Total profit/loss in Crypto: {0} \t({1}%)".format(current_value-invested_amount, round(((current_value-invested_amount)/invested_amount)*100, 2))) 
print("!! Note !! - Skipping the market value of coins that are not listed in WazirX")

In [ ]:
query2